# Lecon 1 : Deviner la suite

> **Bienvenue !** Ce notebook est interactif : tu vas lire,
> executer du code, et ecrire tes propres lignes de Python.

### Comment ca marche ?

1. Clique sur une cellule grise (c'est du code Python)
2. Appuie sur **Shift + Entree** pour l'executer
3. Le resultat s'affiche juste en dessous
4. Passe a la cellule suivante et recommence !

**Regle d'or** : execute les cellules **dans l'ordre**, de haut en bas.
Si tu sautes une cellule, la suivante risque de ne pas marcher.

---

In [ ]:
from IPython.display import HTML, display

_exercices_faits = set()
_NB_TOTAL = 4


def verifier(num_exercice, condition, message_ok, message_aide=""):
    """Valide un exercice avec feedback HTML vert/rouge + compteur."""
    try:
        _result = bool(condition)
    except Exception:
        _result = False
    if _result:
        _exercices_faits.add(num_exercice)
        n = len(_exercices_faits)
        barre = "\U0001f7e9" * n + "\u2b1c" * (_NB_TOTAL - n)
        display(
            HTML(
                f'<div style="padding:10px;background:#d4edda;border-left:5px solid #28a745;'
                f'margin:8px 0;border-radius:4px;font-family:system-ui,-apple-system,sans-serif">'
                f"\u2705 <b>{message_ok}</b><br>"
                f'<span style="color:#555">Progression : {barre} {n}/{_NB_TOTAL}</span></div>'
            )
        )
        if n == _NB_TOTAL:
            display(
                HTML(
                    '<div style="padding:12px;background:linear-gradient(135deg,#3949ab,#6a1b9a);'
                    "color:white;border-radius:8px;text-align:center;font-family:system-ui,-apple-system,sans-serif;"
                    'font-size:1.2em;margin:8px 0">\U0001f3c6 <b>Bravo ! Toutes les activites de cette lecon sont terminees !</b></div>'
                )
            )
    else:
        display(
            HTML(
                f'<div style="padding:10px;background:#fff3cd;border-left:5px solid #ffc107;'
                f'margin:8px 0;border-radius:4px;font-family:system-ui,-apple-system,sans-serif">'
                f"\U0001f4a1 <b>{message_aide}</b></div>"
            )
        )


def exercice(numero, titre, consigne, observation=""):
    """Affiche la banniere d'exercice."""

    def _style_code(text):
        return text.replace(
            "<code>",
            '<code style="font-size:0.95em;background:#bbdefb;'
            'padding:1px 5px;border-radius:3px;font-family:monospace;">',
        )

    obs = ""
    if observation:
        obs = (
            f'<div style="margin-top:6px;color:#555;font-size:0.92em;">'
            f"<b>Ce que tu vas voir\u00a0:</b> {_style_code(observation)}</div>"
        )
    display(
        HTML(
            f'<div style="border-left:5px solid #1565c0;background:#e8f0fe;'
            f"padding:12px 16px; margin:4px 0 10px 0; border-radius:0 8px 8px 0;"
            f'font-family:system-ui,-apple-system,sans-serif; font-size:0.95em;">'
            f'<b style="color:#0d47a1;">Exercice\u00a0{numero} \u2014 {titre}</b><br>'
            f"{_style_code(consigne)}{obs}</div>"
        )
    )


def afficher_heatmap(compteur, titre="Heatmap des bigrammes"):
    """Affiche une heatmap HTML des frequences de bigrammes."""
    lettres = sorted(
        set(k for k in compteur) | set(s for v in compteur.values() for s in v)
    )
    # Find max count for scaling
    max_count = max(c for v in compteur.values() for c in v.values()) if compteur else 1
    header = "".join(
        f'<th style="padding:2px 4px;font-size:0.75em">{letter}</th>'
        for letter in lettres
    )
    rows = ""
    for l1 in lettres:
        row_cells = ""
        for l2 in lettres:
            count = compteur.get(l1, {}).get(l2, 0)
            opacity = count / max_count if max_count > 0 else 0
            bg = f"rgba(102, 126, 234, {opacity:.2f})"
            title_attr = f'title="{l1}->{l2}: {count}"'
            row_cells += f'<td style="padding:2px 4px;background:{bg};text-align:center;font-size:0.65em;border:1px solid #eee;min-width:16px" {title_attr}>{count if count > 0 else ""}</td>'
        rows += f'<tr><th style="padding:2px 4px;font-size:0.75em">{l1}</th>{row_cells}</tr>'
    display(
        HTML(
            f'<!-- tuto-viz --><div style="margin:8px 0;overflow-x:auto"><b>{titre}</b>'
            f'<table style="border-collapse:collapse;margin-top:4px">'
            f"<tr><th></th>{header}</tr>{rows}</table>"
            f'<div style="margin-top:4px;color:#555;font-size:0.8em">Plus la case est foncee, plus la paire est frequente. Survolez pour voir le compte.</div></div>'
        )
    )


def afficher_barres(valeurs, etiquettes, titre="Probabilites"):
    """Affiche des barres horizontales HTML."""
    rows = ""
    max_val = max(valeurs) if valeurs else 1
    for etiq, val in zip(etiquettes, valeurs, strict=False):
        pct = val / max_val * 100 if max_val > 0 else 0
        rows += (
            f'<tr><td style="padding:3px 8px;font-weight:bold;font-size:1em">{etiq}</td>'
            f'<td style="padding:3px;width:300px"><div style="background:linear-gradient(90deg,#3949ab,#6a1b9a);'
            f'width:{max(pct, 2):.0f}%;height:20px;border-radius:4px"></div></td>'
            f'<td style="padding:3px 8px;font-size:0.9em">{val:.0%}</td></tr>'
        )
    display(
        HTML(
            f'<!-- tuto-viz --><div style="margin:8px 0"><b>{titre}</b>'
            f'<table style="border-collapse:collapse;margin-top:4px">{rows}</table></div>'
        )
    )


print("Outils de visualisation charges !")

## Comment une IA "devine" le mot suivant ?

Quand tu ecris un message sur ton telephone, il te **propose** le mot suivant.
Comment fait-il ? Il a appris quels mots viennent souvent apres d'autres.

On va faire pareil, mais avec des **lettres** au lieu de mots.

In [ ]:
exercice(
    1,
    "Quelle lettre suit Q ?",
    "Change <code>ma_reponse</code> ci-dessous, puis <b>Shift + Entree</b>. Quelle lettre vient presque toujours apres Q ?",
    "Si ta reponse est bonne, la box passe au vert.",
)

# ==== MODIFIE ICI ====
ma_reponse = "?"  # <-- Quelle lettre vient apres Q ?
# ======================

# Reponses des autres suites :
print("Reponses :")
print("  A, B, C -> D  (l'alphabet !)")
print("  L, U, N, D -> I  (les jours : Lundi)")
print(f"  Q -> {ma_reponse.upper()}  (ta reponse !)")

# Validation exercice 1
verifier(
    1,
    ma_reponse.lower() == "u",
    "Bravo ! Apres Q vient presque toujours U.",
    "Pas tout a fait... quelle lettre vient presque toujours apres Q ?",
)

---
## Compter les lettres qui suivent

On va prendre une liste de noms de Pokemon et compter quelle lettre
vient apres quelle autre.

Execute la cellule ci-dessous pour charger nos 20 Pokemon :

In [ ]:
# Execute cette cellule pour charger les Pokemon (Shift + Entree)
pokemons = [
    "arcanin",
    "bulbizarre",
    "carapuce",
    "dracaufeu",
    "ectoplasma",
    "evoli",
    "felinferno",
    "gardevoir",
    "goupix",
    "lokhlass",
    "lucario",
    "metamorph",
    "mewtwo",
    "noctali",
    "pikachu",
    "rondoudou",
    "ronflex",
    "salameche",
    "togepi",
    "voltali",
]

print(f"On a {len(pokemons)} Pokemon pour apprendre.")
print()
print("Les 5 premiers :", pokemons[:5])

In [ ]:
exercice(
    2,
    "Explore la liste de Pokemon",
    'Complete les variables ci-dessous avec <code>pokemons[-1]</code> et <code>len("bulbizarre")</code>.',
    "Tu vas voir le dernier Pokemon et le nombre de lettres.",
)

# ==== MODIFIE ICI ====
dernier_pokemon = ...  # <-- Ecris pokemons[-1]
nb_lettres = ...  # <-- Ecris len("bulbizarre")
# ======================

print(f"Le dernier Pokemon est : {dernier_pokemon}")
print(f"'bulbizarre' a {nb_lettres} lettres")

# Validation exercice 2
verifier(
    2,
    isinstance(dernier_pokemon, str)
    and isinstance(nb_lettres, int)
    and dernier_pokemon == pokemons[-1]
    and nb_lettres == len("bulbizarre"),
    f"Bravo ! {dernier_pokemon} est le dernier, et 'bulbizarre' a {nb_lettres} lettres.",
    "Remplace les ... par pokemons[-1] et len('bulbizarre').",
)

---
## Compter les paires de lettres

Maintenant, on va compter **quelle lettre vient apres quelle autre**.
Par exemple, dans "pikachu" :
- apres "p" vient "i"
- apres "i" vient "k"
- apres "k" vient "a"
- ...

On utilise un **point** `.` pour marquer le debut et la fin :
`.pikachu.` signifie "le nom commence et finit ici".

Execute la cellule ci-dessous :

In [ ]:
# Comptons : apres chaque lettre, quelle lettre vient ensuite ?
from collections import Counter

compteur = {}

for pokemon in pokemons:
    mot = "." + pokemon + "."  # ex: ".pikachu."
    for i in range(len(mot) - 1):
        lettre_actuelle = mot[i]
        lettre_suivante = mot[i + 1]
        if lettre_actuelle not in compteur:
            compteur[lettre_actuelle] = Counter()
        compteur[lettre_actuelle][lettre_suivante] += 1

# Que vient-il apres la lettre 'a' ?
print("Apres la lettre 'a', on trouve :")
for lettre, nb in compteur["a"].most_common():
    print(f"  '{lettre}' -> {nb} fois")

# Visualisation : heatmap des bigrammes
afficher_heatmap(compteur, titre="Heatmap : quelle lettre suit quelle autre ?")

In [ ]:
exercice(
    3,
    "Explore les paires de lettres",
    'Change <code>ma_lettre</code> ci-dessous (essaie <code>"p"</code>, <code>"."</code> ou ta lettre preferee).',
    "Les comptages changent selon la lettre choisie.",
)

# ==== MODIFIE ICI ====
ma_lettre = "a"  # <-- Change cette lettre !
# ======================

print(f"Apres la lettre '{ma_lettre}', on trouve :")
for lettre, nb in compteur[ma_lettre].most_common():
    print(f"  '{lettre}' -> {nb} fois")

# Validation exercice 3
verifier(
    3,
    ma_lettre != "a",
    f"Bien joue ! Tu as explore les suites de '{ma_lettre}'.",
    "Change ma_lettre pour une autre lettre, par exemple 'p' ou '.'.",
)

---
## Transformer les comptes en probabilites

Au lieu de dire "la lettre 'r' vient 3 fois apres 'a'",
on veut dire "il y a 25% de chances que 'r' vienne apres 'a'".

C'est ce qu'on appelle une **probabilite** : un nombre entre 0% et 100%
qui dit "a quel point c'est probable".

Execute :

In [ ]:
# Transformer les comptes en probabilites
probas = {}

for lettre, suivantes in compteur.items():
    total = sum(suivantes.values())
    probas[lettre] = {}
    for suivante, nb in suivantes.items():
        probas[lettre][suivante] = nb / total

# Probabilites apres 'a'
print("Probabilites apres 'a' :")
for lettre, p in sorted(probas["a"].items(), key=lambda x: -x[1]):
    barre = "#" * int(p * 40)
    print(f"  '{lettre}' : {p:.0%} {barre}")

# Visualisation : barres de probabilites
_top = sorted(probas["a"].items(), key=lambda x: -x[1])[:8]
afficher_barres(
    [v for _, v in _top], [k for k, _ in _top], titre="Probabilites apres 'a'"
)

In [ ]:
# ---
# ## Generer un nom de Pokemon !
#
# Maintenant on peut **inventer** un nom de Pokemon :
# 1. On part du debut (le point `.`)
# 2. On choisit la lettre suivante au hasard, en respectant les probabilites
# 3. On recommence jusqu'a tomber sur un point `.` (fin du nom)
#
# C'est exactement comme ca que ChatGPT fonctionne, mais avec des **mots**
# au lieu de lettres, et des milliards de parametres au lieu de 20 Pokemon.
#
# Execute :

import random


def generer_pokemon(probas):
    """Genere un nom de Pokemon lettre par lettre."""
    nom = ""
    lettre = "."  # on commence au debut

    while True:
        choix = list(probas[lettre].keys())
        poids = list(probas[lettre].values())
        lettre = random.choices(choix, weights=poids, k=1)[0]

        if lettre == ".":  # fin du nom
            break
        nom += lettre

    return nom


# Generons 10 noms de Pokemon !
print("Noms de Pokemon inventes par notre modele :")
print()
for i in range(10):
    print(f"  {i + 1}. {generer_pokemon(probas).capitalize()}")

In [ ]:
exercice(
    4,
    "Genere plein de Pokemon !",
    "Change <code>nombre</code> ci-dessous (essaie 50 ou 100).",
    "Plus tu en generes, plus tu verras de noms droles.",
)

# ==== MODIFIE ICI ====
nombre = 10  # <-- Mets 50 ici !
# ======================

print(f"Generation de {nombre} Pokemon :")
print()
for i in range(nombre):
    print(f"  {i + 1}. {generer_pokemon(probas).capitalize()}")

# Validation exercice 4
verifier(
    4,
    nombre != 10,
    f"Genial ! Tu as genere {nombre} Pokemon.",
    "Change nombre pour une autre valeur, par exemple 50.",
)

---
## Ce qu'on a appris

- Un modele de langage **predit la suite** en se basant sur ce qu'il a vu avant
- Il utilise des **probabilites** : certaines lettres sont plus probables que d'autres
- Meme un modele tres simple peut generer des noms qui "sonnent" bien

### Le probleme de notre modele

Notre modele ne regarde que **1 lettre en arriere**. Il ne sait pas que
"Pik" est un bon debut de Pokemon, parce qu'il ne voit que la derniere lettre.

Dans la prochaine lecon, on va lui apprendre a **s'ameliorer quand il se trompe**.

---
*Prochaine lecon : [02 - Apprendre de ses erreurs](02_apprendre_des_erreurs.ipynb)*

---

### Sources (ISO 42001)

- **Concept de bigrammes et modeles de langage** : [microgpt.py](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95) -- Andrej Karpathy
- **Approche pedagogique character-level** : [Video "Let's build GPT"](https://www.youtube.com/watch?v=kCc8FmEb1nY) -- Andrej Karpathy (2023)
- **Visualisation des reseaux de neurones** : [3Blue1Brown - Neural Networks](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) -- Grant Sanderson
- **Dataset Pokemon** : (c) Nintendo / Creatures Inc. / GAME FREAK inc., usage educatif. Source : [PokeAPI](https://pokeapi.co/)